In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import xarray as xr

In [ ]:
# Load mask
field_name = 'W2'

mask_path_dict = {
    'W1': 'coadd_W1_1015_40NB_5_arcs.nc',
    'W2': 'coadd_W2_1057_40NB_external_16412.nc',
    'W3': 'coadd_W3_1012_40NB_5_arcs.nc'
}

mask_path = f'/home/alberto/almacen/PAUS_data/masks/{mask_path_dict[field_name]}'
field_mask = xr.open_dataarray(mask_path)

mask_path = f'/home/alberto/almacen/PAUS_data/masks/arclength_{mask_path_dict[field_name]}'
field_arclength_mask = xr.open_dataarray(mask_path)

In [ ]:
cbar = plt.imshow(field_mask.data)
plt.colorbar(cbar)
plt.show()

In [ ]:
plt.hist(field_mask.data.flatten(), bins=100)
plt.show()

In [ ]:
ra_max = field_mask.ra.max()
ra_min = field_mask.ra.min()
dec_max = field_mask.dec.max()
dec_min = field_mask.dec.min()

In [ ]:
# Throw random points in the [RA, DEC] range
N_random_points = 1_000_000

# ra_rand = np.random.uniform(ra_min, ra_max, size=N_random_points)
# dec_rand = np.random.uniform(dec_min, dec_max, size=N_random_points)

def random_uniform_spherical_RADEC(N_random_points, ra_min, ra_max, dec_min, dec_max):
    sintheta_min = np.sin(np.deg2rad(dec_min))
    sintheta_max = np.sin(np.deg2rad(dec_max))

    ra_rand = np.random.uniform(ra_min, ra_max, N_random_points)

    sintheta_rand = np.random.uniform(sintheta_min, sintheta_max, N_random_points)
    dec_rand = np.rad2deg(np.arcsin(sintheta_rand))

    return ra_rand, dec_rand
    
ra_rand, dec_rand = random_uniform_spherical_RADEC(N_random_points, ra_min, ra_max,
                                                   dec_min, dec_max)

In [ ]:
from scipy.interpolate import RectBivariateSpline

# Interpolate random RA,DEC points to the mask
def radec_mask_interpolator(ra_Arr, dec_Arr, mask):
    interpolator = RectBivariateSpline(mask.ra, mask.dec, mask.data.T)
    mask_values = interpolator(ra_Arr, dec_Arr, grid=False)

    masked_radec = (mask_values > 0.9)

    return masked_radec

mask_radec = radec_mask_interpolator(ra_rand, dec_rand, field_mask)

In [ ]:
# Check if the masking is done correctly
plt.scatter(ra_rand[mask_radec], dec_rand[mask_radec], s=0.1)
plt.show()

In [ ]:
##Define the number of regions we want to create,
##as number of division in each of the RA,DEC directions

# Compute more or less equal region areas across fields
area = np.trapz(np.trapz(field_arclength_mask.data,
                         x=field_arclength_mask.y, axis=0),
                x=field_arclength_mask.x)
print(f'Field area (deg^2): {area}')


N_dec = 9
N_ra = 11
N_regions = N_dec * N_ra
print(f'{N_regions=}')
print(f'Region area: {area / N_regions:0.2f}')

In [ ]:
# First divide the field in equal N_ra parts

# Define absolute limits in ra
ra_masked_min = ra_rand[mask_radec].min()
ra_masked_max = ra_rand[mask_radec].max()

# Total number of sources
N_sources = sum(mask_radec)

# Number of sources in every RA region
N_ra_region = N_sources // N_ra

ra_region_lims = []

ra_region_lims.append(ra_masked_min)
for i in range(N_ra - 1):
    ra_region_lims.append(np.sort(ra_rand[mask_radec])[N_ra_region * (i + 1)])
ra_region_lims.append(ra_masked_max)

In [ ]:
# Check RA partition
for i in range(N_ra):
    mask = mask_radec & (ra_rand >= ra_region_lims[i]) & (ra_rand < ra_region_lims[i + 1])
    plt.scatter(ra_rand[mask], dec_rand[mask], s=0.1)
plt.show()

In [ ]:
# Divide in DEC
dec_region_lims = []

for i in range(N_ra):
    mask_ra = mask_radec & (ra_rand >= ra_region_lims[i]) & (ra_rand < ra_region_lims[i + 1])

    this_dec_max = dec_rand[mask_ra].max()
    this_dec_min = dec_rand[mask_ra].min()

    this_dec_region_lims = []

    N_dec_region = sum(mask_ra) // N_dec

    this_dec_region_lims.append(this_dec_min)
    for j in range(N_dec - 1):
        this_dec_region_lims.append(np.sort(dec_rand[mask_ra])[N_dec_region * (j + 1)])
    this_dec_region_lims.append(this_dec_max)

    dec_region_lims.append(this_dec_region_lims)
    

In [ ]:
# Check final partition

for i in range(N_ra):
    mask_ra = mask_radec & (ra_rand >= ra_region_lims[i]) & (ra_rand < ra_region_lims[i + 1])
    for j in range(N_dec):
        mask = mask_ra & (dec_rand >= dec_region_lims[i][j]) & (dec_rand < dec_region_lims[i][j + 1])
        plt.scatter(ra_rand[mask], dec_rand[mask], s=0.1)
plt.show()

In [ ]:
from load_paus_cat import load_paus_cat

# Load PAUS
path_to_paus_cat = [f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_3arcsec_{field_name}_extinction_corrected.pq']
paus_cat = load_paus_cat(path_to_paus_cat)
mask_NB_number = (paus_cat['NB_number'] > 39)
paus_cat['flx'] = paus_cat['flx'][:, mask_NB_number]
paus_cat['err'] = paus_cat['err'][:, mask_NB_number]
paus_cat['NB_mask'] = paus_cat['NB_mask'][:, mask_NB_number]
for key in paus_cat.keys():
    if key in ['flx', 'err', 'NB_mask', 'area']:
        continue
    paus_cat[key] = paus_cat[key][mask_NB_number]

In [ ]:
# Compute NB>=40 incompletion
N_40 = sum(paus_cat['NB_number'] == 40)
mask_30 = (paus_cat['NB_number'] >= 30)
N_30 = sum(radec_mask_interpolator(paus_cat['RA'][mask_30], paus_cat['DEC'][mask_30], field_mask))
N_40/N_30

In [ ]:
# Assign region IDs
reg_id_Arr = np.ones_like(paus_cat['ref_id']).astype(int)

fig, ax = plt.subplots(figsize=(16, 10))

for i in range(N_ra):
    mask_ra = mask_radec & (ra_rand >= ra_region_lims[i]) & (ra_rand < ra_region_lims[i + 1])
    for j in range(N_dec):
        mask = mask_ra & (dec_rand >= dec_region_lims[i][j]) & (dec_rand < dec_region_lims[i][j + 1])
        ax.scatter(ra_rand[mask], dec_rand[mask], s=0.4, alpha=0.5, color='k')

reg_id = 0
for i in range(N_ra):
    mask_ra = (paus_cat['RA'] >= ra_region_lims[i]) & (paus_cat['RA'] < ra_region_lims[i + 1])
    for j in range(N_dec):
        mask = mask_ra & (paus_cat['DEC'] >= dec_region_lims[i][j]) & (paus_cat['DEC'] < dec_region_lims[i][j + 1])
        ax.scatter(paus_cat['RA'][mask], paus_cat['DEC'][mask], s=0.1)

        reg_id_Arr[mask] = reg_id
        reg_id += 1

ax.set_xlabel('RA')
ax.set_ylabel('DEC')

ax.set_aspect('equal', adjustable='box')

plt.show()

In [ ]:
np.save(f'/home/alberto/almacen/PAUS_data/masks/reg_id_Arr_{field_name}', reg_id_Arr)